In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from DSA import DSA, GeneralizedDSA, InputDSA
from DSA import DMD, DMDc, SubspaceDMDc, ControllabilitySimilarityTransformDist
from DSA import DMDConfig, DMDcConfig, SubspaceDMDcConfig
from DSA import SimilarityTransformDistConfig, ControllabilitySimilarityTransformDistConfig
from pydmd import DMD as pDMD
import DSA.pykoopman as pk
%load_ext autoreload
%autoreload 2


In [ ]:
%pdb

Automatic pdb calling has been turned ON


In [ ]:
d1 = np.random.random(size=(20,5))
u1 = np.random.random(size=(20,2))

d2 = np.random.random(size=(3,20,5))
u2 = np.random.random(size=(3,20,2))

d3 = [np.random.random(size=(i,20,5)) for i in range(1,10)]
u3 = [np.random.random(size=(i,20,2)) for i in range(1,10)]

d4 = [np.random.random(size=(i+5,5)) for i in range(1,10)]
u4 = [np.random.random(size=(i+5,2)) for i in range(1,10)]

d5 = d4 + d3
u5 = u4 + u3

len(d5), len(d4)

(18, 9)

In [ ]:

# dmdc = DMDc(d5,u5,n_delays=2,rank_input=10,rank_output=10)
# dmdc.fit()
# print(dmdc.A_v.shape)
# print(dmdc.B_v.shape)

# subdmdc = SubspaceDMDc(d3,u3,n_delays=3,rank=5,backend='n4sid')
# subdmdc.fit()
# print(subdmdc.A_v.shape)
# print(subdmdc.B_v.shape)


#TODO: fix this case
subdmdc = SubspaceDMDc(d2,u2,n_delays=10,rank=5,backend='n4sid')
subdmdc.fit()
print(subdmdc.A_v.shape)
print(subdmdc.B_v.shape)


#TODO: fix this case
# subdmdc = SubspaceDMDc(d3,u3,n_delays=2,rank=10,backend='n4sid')
# subdmdc.fit()
# print(subdmdc.A_v.shape)
# print(subdmdc.B_v.shape)

#TODO: check predictions for all cases

ValueError: Trial 0: y and u have different time lengths

> /Users/mitchellostrow/Desktop/Projects/DSAv2/DSAPublic2/DSA/DSA/subspace_dmdc.py(98)_validate_and_collect_data()
     96                 raise ValueError(f"Trial {i}: u has {u_trial.shape[0]} inputs, expected {m}")
     97             if y_trial.shape[1] != u_trial.shape[1]:
---> 98                 raise ValueError(f"Trial {i}: y and u have different time lengths")
     99 
    100         U_p_all = []



In [12]:
d1s = [np.random.random(size=(1+_,20,5)) for _ in range(3)]
u1s = [np.random.random(size=(1+_,20,2)) for _ in range(3)]

d3s = [d3 for _ in range(3)]
u3s = [u3 for _ in range(3)]

dmdconfig = DMDConfig(n_delays=20)
simdistconfig = SimilarityTransformDistConfig(score_method='wasserstein')
csimdistconfig = ControllabilitySimilarityTransformDistConfig(compare='joint',
    score_method='euclidean', align_inputs=False,return_distance_components=True)
#works
# dsa = DSA(d1s,dmd_class=pk.Koopman,
#     observables=pk.observables.TimeDelay(),regressor=pDMD(svd_rank=5),
#     score_method='wasserstein')
dmd_config = SubspaceDMDcConfig(rank=5)
dmdc_config = DMDcConfig()

dsa = InputDSA(d3s,u3s,simdist_config=csimdistconfig,
    dmd_class=DMDc,dmd_config=dmdc_config,verbose=True)
sim = dsa.fit_score()
print(sim.shape)

#fixed
dsa = GeneralizedDSA(d3s,X_control=u3s,
    dmd_class=DMDc,dmd_config=dict(n_delays=5,rank_input=5,rank_output=5),
    verbose=True)
sim = dsa.fit_score()
sim.shape

#TODO: check generalized dsa with other data structures for data and inputs
#TODO: check generalized dsa with the other comparison metric and changing the config


Computing DMD similarities: 100%|██████████| 3/3 [00:00<00:00, 904.46it/s]
/Users/mitchellostrow/Desktop/Projects/DSAv2/DSAPublic2/DSA/DSA/dsa.py:408: UserWarning: Warning: You are using a DMD model that fits a control operator but comparing with a DSA metric that does not compare control operators
  warnings.warn(


(3, 3, 3)


Computing DMD similarities:  33%|███▎      | 1/3 [00:03<00:06,  3.07s/it]

KeyboardInterrupt: 

> /Users/mitchellostrow/Desktop/Projects/DSAv2/DSAPublic2/DSA/DSA/simdist.py(128)forward()
    126     def forward(self, X):
    127         # (I + X)(I - X)^{-1}
--> 128         return torch.linalg.solve(self.Id + X, self.Id - X)
    129 
    130 

